# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
import json

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key
#print(geoapify_key)

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,34.41,98,100,10.92,GS,1689959927
1,1,mount pearl,47.5166,-52.7813,84.45,62,20,17.27,CA,1689959904
2,2,mtwara,-10.2667,40.1833,79.14,56,100,13.33,TZ,1689960056
3,3,olonkinbyen,70.9221,-8.7187,41.38,79,100,4.45,SJ,1689959729
4,4,kodiak,57.7900,-152.4072,53.11,82,0,0.00,US,1689959999


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
geo_data_frame = gpd.GeoDataFrame(city_data_df, geometry=gpd.points_from_xy(city_data_df.Lng, city_data_df.Lat))

def hover_tool(row):
    return f"<h5>{row['City']}</h5><p>Humidity: {row['Humidity']}</p>"


# Display the map
map_plot = geo_data_frame.hvplot(geo=True, tiles='EsriTerrain', hover_cols=['City', 'Humidity'], size='Humidity', color='City',
                      line_color='green', alpha=0.7)


# Display the map plot_1
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

mid_humidity = city_data_df[(city_data_df["Humidity"] > 50) & (city_data_df["Humidity"] < 70)]
low_wind = mid_humidity.loc[mid_humidity["Wind Speed"] < 10]
low_cloud = low_wind.loc[low_wind["Cloudiness"] > 20]
temp_range_df = low_cloud[(low_cloud["Max Temp"] > 75) & (low_cloud["Max Temp"] < 85)]

# Drop any rows with null values
temp_range_df = temp_range_df.dropna()

# Display sample data
temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,geometry
6,6,roma,43.2128,-75.4557,75.72,60,100,5.75,US,1689959839,POINT (-75.45570 43.21280)
190,190,carutapera,-1.1950,-46.0200,82.62,66,26,9.37,BR,1689960245,POINT (-46.02000 -1.19500)
301,301,bosanska krupa,44.8825,16.1514,83.97,67,27,0.45,BA,1689960113,POINT (16.15140 44.88250)
339,339,boende,-0.2167,20.8667,83.30,55,92,0.60,CD,1689960012,POINT (20.86670 -0.21670)
351,351,bardai,11.6500,7.2333,79.45,57,100,3.42,NG,1689960120,POINT (7.23330 11.65000)
355,355,ad-damazin,11.7891,34.3592,81.36,69,89,9.26,SD,1689960121,POINT (34.35920 11.78910)
374,374,lisala,2.1500,21.5167,80.73,68,92,2.51,CD,1689960125,POINT (21.51670 2.15000)
375,375,ituango,7.1712,-75.7640,79.65,51,100,2.46,CO,1689960125,POINT (-75.76400 7.17120)
402,402,el parral,16.3675,-93.0058,83.68,60,72,1.61,MX,1689960132,POINT (-93.00580 16.36750)
452,452,brookings,44.3114,-96.7984,77.18,63,28,4.90,US,1689960046,POINT (-96.79840 44.31140)


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = temp_range_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,roma,US,43.2128,-75.4557,60,
190,carutapera,BR,-1.1950,-46.0200,66,
301,bosanska krupa,BA,44.8825,16.1514,67,
339,boende,CD,-0.2167,20.8667,55,
351,bardai,NG,11.6500,7.2333,57,
355,ad-damazin,SD,11.7891,34.3592,69,
374,lisala,CD,2.1500,21.5167,68,
375,ituango,CO,7.1712,-75.7640,51,
402,el parral,MX,16.3675,-93.0058,60,
452,brookings,US,44.3114,-96.7984,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

#params = {"categories": categories, "apikey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}" 
      
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
   

Starting hotel search
roma - nearest hotel: Wingate by Wyndham Rome
carutapera - nearest hotel: No hotel found
bosanska krupa - nearest hotel: Hotel "Stari grad"
boende - nearest hotel: Hotel Reference
bardai - nearest hotel: No hotel found
ad-damazin - nearest hotel: No hotel found
lisala - nearest hotel: No hotel found
ituango - nearest hotel: Palacio Municipal - Hotel Ituango
el parral - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
tchibanga - nearest hotel: Auberge Nyanga
boa vista - nearest hotel: Aipana Plaza Hotel
porto torres - nearest hotel: Hotel Elisa


,City,Country,Lat,Lng,Humidity,Hotel Name
6,roma,US,43.2128,-75.4557,60,Wingate by Wyndham Rome
190,carutapera,BR,-1.1950,-46.0200,66,No hotel found
301,bosanska krupa,BA,44.8825,16.1514,67,"Hotel ""Stari grad"""
339,boende,CD,-0.2167,20.8667,55,Hotel Reference
351,bardai,NG,11.6500,7.2333,57,No hotel found
355,ad-damazin,SD,11.7891,34.3592,69,No hotel found
374,lisala,CD,2.1500,21.5167,68,No hotel found
375,ituango,CO,7.1712,-75.7640,51,Palacio Municipal - Hotel Ituango
402,el parral,MX,16.3675,-93.0058,60,No hotel found
452,brookings,US,44.3114,-96.7984,63,Quality Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display


geo_data_frame = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df.Lng, hotel_df.Lat))

def hover_tool(row):
    return f"<h5>{row['City']}</h5><p>Hotel Name: {row['Hotel Name']}</p>"


# Display the map
map_plot_2 = geo_data_frame.hvplot(geo=True, tiles='EsriTerrain', hover_cols=['City', 'Hotel Name'], size='Humidity', color='City',
                      line_color='green', alpha=0.7)

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name)